In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_3_Bin_Packing/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-21 14:45:16.401130


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Here is a mathematical optimization model for your problem:

Parameters:
- Let $I$ be the set of items.
- Let $B$ be the set of boxes.
- Let $w_i$ be the weight of item $i \in I$.
- Let $c_b$ be the capacity of box $b \in B$.
- Let $M$ be the upper bound on the number of boxes used.

Decision Variables:
- Let $x_{ib}$ be a binary variable equal to 1 if item $i$ is packed in box $b$, and 0 otherwise.
- Let $y_b$ be a binary variable equal to 1 if box $b$ is used, and 0 otherwise.

Objective Function:
Minimize the number of boxes used: $\min \sum_{b \in B} y_b$.

Constraints:
1. Each item is packed in exactly one box: $\sum_{b \in B} x_{ib} = 1, \quad \forall i \in I$.
2. The total weight of items in each box does not exceed the box capacity: $\sum_{i \in I} w_i \cdot x_{ib} \leq c_b \cdot y_b, \quad \forall b \in B$.
3. The number of boxes used does not exceed the upper bound: $\sum_{b \in B} y_b \leq M$.
4. If an item is packed in a box, then the box is used: $x_{ib} \leq y_b, \quad \forall i \in I, b \in B$.
5. Binary restrictions on decision variables: $x_{ib} \in \{0, 1\}, \quad \forall i \in I, b \in B$, and $y_b \in \{0, 1\}, \quad \forall b \in B$.

This formulation is a binary integer linear programming problem, which can be solved using standard optimization software.

In [6]:
print(response_1_text)

Here is a mathematical optimization model for your problem:

Parameters:
- Let $I$ be the set of items.
- Let $B$ be the set of boxes.
- Let $w_i$ be the weight of item $i \in I$.
- Let $c_b$ be the capacity of box $b \in B$.
- Let $M$ be the upper bound on the number of boxes used.

Decision Variables:
- Let $x_{ib}$ be a binary variable equal to 1 if item $i$ is packed in box $b$, and 0 otherwise.
- Let $y_b$ be a binary variable equal to 1 if box $b$ is used, and 0 otherwise.

Objective Function:
Minimize the number of boxes used: $\min \sum_{b \in B} y_b$.

Constraints:
1. Each item is packed in exactly one box: $\sum_{b \in B} x_{ib} = 1, \quad \forall i \in I$.
2. The total weight of items in each box does not exceed the box capacity: $\sum_{i \in I} w_i \cdot x_{ib} \leq c_b \cdot y_b, \quad \forall b \in B$.
3. The number of boxes used does not exceed the upper bound: $\sum_{b \in B} y_b \leq M$.
4. If an item is packed in a box, then the box is used: $x_{ib} \leq y_b, \quad \f

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
import pyomo.environ as pyo

# Sample data (replace with your own data)
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w = {'item1': 5, 'item2': 3, 'item3': 4}  # Weight of items
c = {'box1': 7, 'box2': 6, 'box3': 8}  # Capacity of boxes
M = 2  # Upper bound on the number of boxes used

model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.B = pyo.Set(initialize=B)

# Parameters
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.B, initialize=c)
model.M = pyo.Param(initialize=M)

# Decision Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)
model.y = pyo.Var(model.B, domain=pyo.Binary)

# Objective Function
def obj_rule(model):
    return pyo.summation(model.y)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def constraint1(model, i):
    return pyo.summation(model.x[i, b] for b in model.B) == 1

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model, b):
    return pyo.summation(model.w[i] * model.x[i, b] for i in model.I) <= model.c[b] * model.y[b]

model.constraint2 = pyo.Constraint(model.B, rule=constraint2)

def constraint3(model):
    return pyo.summation(model.y) <= model.M

model.constraint3 = pyo.Constraint(rule=constraint3)

def constraint4(model, i, b):
    return model.x[i, b] <= model.y[b]

model.constraint4 = pyo.Constraint(model.I, model.B, rule=constraint4)

# Solve the problem
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Print the results
if result.solver.status == pyo.SolverStatus.ok:
    print("Objective value:", pyo.value(model.obj))
    print("x values:")
    for i in model.I:
        for b in model.B:
            if pyo.value(model.x[i, b]) > 0:
                print(f"Item {i} is packed in box {b}")
    print("y values:")
    for b in model.B:
        if pyo.value(model.y[b]) > 0:
            print(f"Box {b} is used")
else:
    print("Failed to solve the problem.")
```

This code uses the GLPK solver, which is an open-source linear programming solver. You can replace the solver with other Pyomo-compatible solvers such as CPLEX, Gurobi, or CBC. Make sure to replace the sample data with your own data.

In [10]:
print(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
import pyomo.environ as pyo

# Sample data (replace with your own data)
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w = {'item1': 5, 'item2': 3, 'item3': 4}  # Weight of items
c = {'box1': 7, 'box2': 6, 'box3': 8}  # Capacity of boxes
M = 2  # Upper bound on the number of boxes used

model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.B = pyo.Set(initialize=B)

# Parameters
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.B, initialize=c)
model.M = pyo.Param(initialize=M)

# Decision Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)
model.y = pyo.Var(model.B, domain=pyo.Binary)

# Objective Function
def obj_rule(model):
    return pyo.summation(model.y)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def constraint1(model, i):
    return pyo.summation(model.x[i, b] for b in model.B) ==

### Code Executability

In [18]:
import pyomo.environ as pyo

# Sample data (replace with your own data)
I = ['item1', 'item2', 'item3']  # Set of items
B = ['box1', 'box2', 'box3']  # Set of boxes
w = {'item1': 5, 'item2': 3, 'item3': 4}  # Weight of items
c = {'box1': 7, 'box2': 6, 'box3': 8}  # Capacity of boxes
M = 2  # Upper bound on the number of boxes used

model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.B = pyo.Set(initialize=B)

# Parameters
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.B, initialize=c)
model.M = pyo.Param(initialize=M)

# Decision Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)
model.y = pyo.Var(model.B, domain=pyo.Binary)

# Objective Function
def obj_rule(model):
    return pyo.summation(model.y)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def constraint1(model, i):
    return pyo.summation(model.x[i, b] for b in model.B) == 1

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model, b):
    return pyo.summation(model.w[i] * model.x[i, b] for i in model.I) <= model.c[b] * model.y[b]

model.constraint2 = pyo.Constraint(model.B, rule=constraint2)

def constraint3(model):
    return pyo.summation(model.y) <= model.M

model.constraint3 = pyo.Constraint(rule=constraint3)

def constraint4(model, i, b):
    return model.x[i, b] <= model.y[b]

model.constraint4 = pyo.Constraint(model.I, model.B, rule=constraint4)

# Solve the problem
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Print the results
if result.solver.status == pyo.SolverStatus.ok:
    print("Objective value:", pyo.value(model.obj))
    print("x values:")
    for i in model.I:
        for b in model.B:
            if pyo.value(model.x[i, b]) > 0:
                print(f"Item {i} is packed in box {b}")
    print("y values:")
    for b in model.B:
        if pyo.value(model.y[b]) > 0:
            print(f"Box {b} is used")
else:
    print("Failed to solve the problem.")

2024-05-21 15:06:39,649 ERROR pyomo.core: Rule failed when generating expression for Constraint constraint1 with index item1:
ValueError: Error executing sum_product(): The last argument value must be a variable or expression object if no 'index' option is specified
2024-05-21 15:06:39,649 ERROR pyomo.core: Constructing component 'constraint1' from data=None failed:
ValueError: Error executing sum_product(): The last argument value must be a variable or expression object if no 'index' option is specified


ValueError: Error executing sum_product(): The last argument value must be a variable or expression object if no 'index' option is specified

### Solution Correctness

In [22]:
import pyomo.environ as pyo

B = 9
w = [2,3,4,5,6,7,8]
q = [4,2,6,6,2,2,2]
s=[]
for j in range(len(w)):
    for i in range(q[j]):
        s.append(w[j])

def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol

n = len(s)
U = len(FFD(s, B))

# Sample data (replace with your own data)
I = [i for i in range(n)]  # Set of items
B = [i for i in range(U)]  # Set of boxes
w = {i : v for i, v in enumerate(s)} # Weight of items
c = {i : B for i in range(U)}  # Capacity of boxes
M = len(B)  # Upper bound on the number of boxes used

model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=I)
model.B = pyo.Set(initialize=B)

# Parameters
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.B, initialize=c)
model.M = pyo.Param(initialize=M)

# Decision Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)
model.y = pyo.Var(model.B, domain=pyo.Binary)

# Objective Function
def obj_rule(model):
    return pyo.summation(model.y)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def constraint1(model, i):
    return sum(model.x[i, b] for b in model.B) == 1

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model, b):
    return sum(model.w[i] * model.x[i, b] for i in model.I) <= model.c[b] * model.y[b]

model.constraint2 = pyo.Constraint(model.B, rule=constraint2)

def constraint3(model):
    return sum(model.y[b] for b in model.B) <= model.M

model.constraint3 = pyo.Constraint(rule=constraint3)

def constraint4(model, i, b):
    return model.x[i, b] <= model.y[b]

model.constraint4 = pyo.Constraint(model.I, model.B, rule=constraint4)

# Solve the problem
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Print the results
if result.solver.status == pyo.SolverStatus.ok:
    print("Objective value:", pyo.value(model.obj))
    print("x values:")
    for i in model.I:
        for b in model.B:
            if pyo.value(model.x[i, b]) > 0:
                print(f"Item {i} is packed in box {b}")
    print("y values:")
    for b in model.B:
        if pyo.value(model.y[b]) > 0:
            print(f"Box {b} is used")
else:
    print("Failed to solve the problem.")

2024-05-21 15:09:37,487 ERROR pyomo.core: Rule failed when generating expression for Constraint constraint2 with index 0:
AttributeError: 'list' object has no attribute 'is_expression_type'
2024-05-21 15:09:37,487 ERROR pyomo.core: Constructing component 'constraint2' from data=None failed:
AttributeError: 'list' object has no attribute 'is_expression_type'


AttributeError: 'list' object has no attribute 'is_expression_type'

In [23]:
import pyomo.environ as pyo

# Bin capacity
bin_capacity = 9

# Weights of different types of items
weights = [2, 3, 4, 5, 6, 7, 8]

# Quantities of each type of item
quantities = [4, 2, 6, 6, 2, 2, 2]

# List of individual items
items = []
for j in range(len(weights)):
    for i in range(quantities[j]):
        items.append(weights[j])

# First-Fit Decreasing (FFD) Heuristic
def FFD(items, bin_capacity):
    remain = [bin_capacity]
    solution = [[]]
    for item in sorted(items, reverse=True):
        for j, free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                solution[j].append(item)
                break
        else:
            solution.append([item])
            remain.append(bin_capacity - item)
    return solution

# Upper bound on the number of bins
upper_bound_bins = len(FFD(items, bin_capacity))

# Pyomo model
model = pyo.ConcreteModel()

# Sets and Parameters
num_items = len(items)
item_indices = list(range(num_items))
bin_indices = list(range(upper_bound_bins))
weights_dict = {i: v for i, v in enumerate(items)}
capacities_dict = {i: bin_capacity for i in range(upper_bound_bins)}
max_bins = upper_bound_bins

model.I = pyo.Set(initialize=item_indices)
model.B = pyo.Set(initialize=bin_indices)
model.w = pyo.Param(model.I, initialize=weights_dict)
model.c = pyo.Param(model.B, initialize=capacities_dict)
model.M = pyo.Param(initialize=max_bins)

# Decision Variables
model.x = pyo.Var(model.I, model.B, domain=pyo.Binary)
model.y = pyo.Var(model.B, domain=pyo.Binary)

# Objective Function
def obj_rule(model):
    return pyo.summation(model.y)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def constraint1(model, i):
    return sum(model.x[i, b] for b in model.B) == 1

model.constraint1 = pyo.Constraint(model.I, rule=constraint1)

def constraint2(model, b):
    return sum(model.w[i] * model.x[i, b] for i in model.I) <= model.c[b] * model.y[b]

model.constraint2 = pyo.Constraint(model.B, rule=constraint2)

def constraint3(model):
    return sum(model.y[b] for b in model.B) <= model.M

model.constraint3 = pyo.Constraint(rule=constraint3)

def constraint4(model, i, b):
    return model.x[i, b] <= model.y[b]

model.constraint4 = pyo.Constraint(model.I, model.B, rule=constraint4)

# Solve the problem
solver = pyo.SolverFactory('glpk')
result = solver.solve(model)

# Print the results
if result.solver.status == pyo.SolverStatus.ok:
    print("Objective value:", pyo.value(model.obj))
    print("x values:")
    for i in model.I:
        for b in model.B:
            if pyo.value(model.x[i, b]) > 0:
                print(f"Item {i} is packed in box {b}")
    print("y values:")
    for b in model.B:
        if pyo.value(model.y[b]) > 0:
            print(f"Box {b} is used")
else:
    print("Failed to solve the problem.")


Objective value: 13.0
x values:
Item 0 is packed in box 10
Item 1 is packed in box 8
Item 2 is packed in box 12
Item 3 is packed in box 0
Item 4 is packed in box 3
Item 5 is packed in box 12
Item 6 is packed in box 12
Item 7 is packed in box 9
Item 8 is packed in box 7
Item 9 is packed in box 5
Item 10 is packed in box 4
Item 11 is packed in box 2
Item 12 is packed in box 5
Item 13 is packed in box 1
Item 14 is packed in box 7
Item 15 is packed in box 2
Item 16 is packed in box 9
Item 17 is packed in box 4
Item 18 is packed in box 0
Item 19 is packed in box 3
Item 20 is packed in box 10
Item 21 is packed in box 8
Item 22 is packed in box 6
Item 23 is packed in box 11
y values:
Box 0 is used
Box 1 is used
Box 2 is used
Box 3 is used
Box 4 is used
Box 5 is used
Box 6 is used
Box 7 is used
Box 8 is used
Box 9 is used
Box 10 is used
Box 11 is used
Box 12 is used
